## TL classifier 

In [1]:
import numpy as np
import tensorflow as tf

from PIL import Image

In [2]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

In [3]:
# label map and computation graph path
category_index = {1: {'id': 1, 'name': 'Green'}, 
                  2: {'id': 2, 'name': 'Red'}, 
                  3: {'id': 3, 'name': 'Yellow'}, 
                  4: {'id': 4, 'name': 'off'}}

faster_rcnn_real_model = './frozen_inference_graph.pb'
class TLClassifier(object):
    def __init__(self, model_graph_path=faster_rcnn_real_model):
        #TODO load classifier
        self.detection_graph = tf.Graph()
        with self.detection_graph.as_default():
            od_graph_def = tf.GraphDef()
            
            # read serialized graph and load it into the detection graph
            with tf.gfile.GFile(model_graph_path, 'rb') as fid:    
                serialized_graph = fid.read()
                od_graph_def.ParseFromString(serialized_graph)
                tf.import_graph_def(od_graph_def, name='')

            # Define input and output Tensors for detection_graph
            self.image_tensor = self.detection_graph.get_tensor_by_name('image_tensor:0')

            # class labels with the corresponding confidence level.
            self.scores_tensor = self.detection_graph.get_tensor_by_name('detection_scores:0')
            self.classes_tensor = self.detection_graph.get_tensor_by_name('detection_classes:0')
        
        # start tf session with the detection graph
        self.sess = tf.Session(graph=self.detection_graph)
        
    def get_classification(self, image):
        """Determines the color of the traffic light in the image

        Args:
            image (cv::Mat): image containing the traffic light

        Returns:
            int: ID of traffic light color (specified in styx_msgs/TrafficLight)

        """
        #TODO implement light color prediction
        image_np = load_image_into_numpy_array(image)
        # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(image_np, axis=0)
              
        # Actual detection.
        (scores, classes) = self.sess.run(
              [self.scores_tensor, self.classes_tensor],
              feed_dict={self.image_tensor: image_np_expanded})

        scores = np.squeeze(scores)
        classes = np.squeeze(classes).astype(np.int32)

        return category_index[classes[np.argmax(scores)]]['name']
        

## classification examples

In [4]:
import time
import os

from glob import glob

In [5]:
PATH_TO_TEST_IMAGES_DIR = './test_imgs'

print(os.path.join(PATH_TO_TEST_IMAGES_DIR, '*.jpg'))
TEST_IMAGE_PATHS = glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, '*.jpg'))
print("Length of test images:", len(TEST_IMAGE_PATHS))

./test_imgs\*.jpg
Length of test images: 10


In [6]:
tl_detector = TLClassifier()

In [7]:
image = Image.open(TEST_IMAGE_PATHS[1])
time0 = time.time()
traffic_light = tl_detector.get_classification(image)
time1 = time.time()
print("Time in milliseconds", (time1 - time0) * 1000, "\n") 
print(traffic_light)

Time in milliseconds 6733.838081359863 

Red


In [8]:
image = Image.open(TEST_IMAGE_PATHS[2])
time0 = time.time()
traffic_light = tl_detector.get_classification(image)
time1 = time.time()
print("Time in milliseconds", (time1 - time0) * 1000, "\n") 
print(traffic_light)

Time in milliseconds 1184.0391159057617 

Green


In [9]:
image = Image.open(TEST_IMAGE_PATHS[3])
time0 = time.time()
traffic_light = tl_detector.get_classification(image)
time1 = time.time()
print("Time in milliseconds", (time1 - time0) * 1000, "\n") 
print(traffic_light)

Time in milliseconds 2368.5338497161865 

Red


In [10]:
image = Image.open(TEST_IMAGE_PATHS[4])
time0 = time.time()
traffic_light = tl_detector.get_classification(image)
time1 = time.time()
print("Time in milliseconds", (time1 - time0) * 1000, "\n") 
print(traffic_light)

Time in milliseconds 1932.8627586364746 

Green
